# "Remaining Useful Life Estimation in Prognostics Using Deep Convolution Neural Networks" by Xiang Li et al.

This document reproduces the implementation of a Deep LSTM Network  by Xiang Li et al. applied to the NASA "CMAPSS" dataset. This implementation has been done in Keras.

Copyright (c) by Manuel Arias, Christian Schneebeli and Lukas R. Peter 2017-12-01.

Before jumping into the problem, let's run the cell below to load the required packages.

In [20]:
import os
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import Dropout
from keras.layers import Input, LSTM, RepeatVector
from keras.models import Model
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.initializers import glorot_uniform
from keras import optimizers
from IPython.display import SVG, clear_output
from keras.utils import plot_model
from os import path

import keras.callbacks
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt

get_ipython().magic(u'matplotlib inline')

In [21]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

# 1  "CMAPSS" Dataset

The C-MAPSS dataset FD001 from NASA was used to evaluate our model. This dataset provides degradation trajectories of 100 turbofan engines with unknown and different initial health condition for one operative set-point and one failure mode. The data were synthetically generated with the Commercial Modular Aero-Propulsion System Simulation (C-MAPSS) dynamical model. The training data contains multivariate sensors readings of the complete run-to-failure trajectories. Therefore, the records stop at the cycle/time the engine failed. For the test set truncated time-series of various lengths prior to failure are provided for 100 engines. A total number of 20k and 13k cycles are available for the training and test set respectively.

**Details of the "CMAPSS" dataset**
- Training: 17731 inputs from 100 engine trajectories. It uses a sliding time window of 30 time stamps. 
- Test: 100 points from 100 engine trajectories. It takes the last available 30 time stamps.

# 1.1  Data Pre-processing

As proposed in Li, we follow a three-step process to create the input sequences to LSTM and CNN models. Since the FD001 dataset is limited to one operative condition, 10 out of the 24 sensors show constant values. Therefore, we first dropped these values and we normalized the other 14 sensors by min/max-normalization to a range $[-1, 1]$. Second, the original dataset was processed with a sliding time window approach of size $N_f = 30$ and stride of 1. The sliding window means that the first input sample to the network takes measurements from cycles 1-30, the second 2-31, the third 3-32, and so on for each unit of the fleet. The RUL label for a sample is then simply the total number of cycles of the engine is able to operate minus the cycle where the window ends. As discussed in Li, we use a window size of 30 cycles/times since the smallest test sample consists of 30 cycles. This approach provides 17731 training samples. Lastly, the maximum horizon of prediction for RUL i.e $R_{early}$ was limited to 125 cycles following the standard procedure adopted by other researchers Li, Malhotra. This has a noticeable impact on the model accuracy and makes our models more stable. From a practical point of view, it implies that we are not interested in prediction RUL further away than 125 cycles ahead.

**Time window details**
- $N_{tw} = 30$ 
- Stride = 1

In [22]:
N_tw     = 30                                                               # Time Window (N_tw)
R_early  = 125                                                              # Max RUL in training set
stride   = 1
sel      = np.array([6, 7, 8, 11, 12, 13, 15, 16, 17, 18, 19, 21, 24, 25])  # Index of input features

In [23]:
def sliding_window(data, N_tw = 30, stride = 1):
    N_en = np.unique(data[:,0]).shape[0]                            # Number of engines (N_en)
    m = 0
    for i in range(N_en):
        n_H   = data[data[:,0] == i+1,0].shape[0]
        N_sw  = int((n_H- N_tw) / stride + 1)                       # Number of sliding windows for engine 'i' 
        for h in range(N_sw):
            m = m + 1    
    return m, N_en            

In [24]:
def load_dataset(N_tw, stride, sel, R_early):
    # Load training data
    basepath        = path.dirname(os.getcwd()) 
    train_set       = np.loadtxt("train_FD001.txt")  # Training set
    train_set_x_org = train_set[:,sel]                              # Training set input space (x)    
    train_set_c     = train_set[:,np.array([1])]                    # Training set cycles (c)
    
    # Normalize the data
    ub = train_set_x_org.max(0)
    lb = train_set_x_org.min(0)    
    train_set_x = 2 * (train_set_x_org - lb) / (ub - lb) - 1   
   
    N_ft    = sel.shape[0]                                           # Nunber of features (N_ft)
    m, N_en = sliding_window(train_set, N_tw, stride)                # Number of training data & engines
    
    train_x = np.empty((m, N_tw, N_ft), float)
    train_y = np.empty((m), float)
    
    k = 0
    for i in range(N_en):
        idx       = train_set[:,0] == i+1                            # Index for engine number 'i'
        train_i_x = train_set_x[idx,:]                               # Engine 'i' training  data
        train_i_c = train_set_c[idx]                                 # Engine 'i' cycles (c)
        train_i_y = train_i_c[-1] - train_i_c                        # RUL: Remaining Useful Lifetime for engine 'i'
        train_i_y[train_i_y > R_early] = R_early                     # R_early = 125
        N_sw      = int((train_i_x.shape[0] - N_tw) / stride + 1)    # Number of sliding windows for engine 'i' 
        for h in range(N_sw):
            k = k + 1
            vert_start = h * stride
            vert_end   = h * stride + N_tw
            train_i_x_slice = train_i_x[vert_start:vert_end,:]       # Training input data for engine 'i' on time window 'h'
            train_i_y_slice = train_i_y[vert_end-1,:]                # Training output data for engine 'i' on time window 'h'
            train_i_x_slice.shape = (N_tw, N_ft)                  # Reshape training set input (N_tw, N_ft, 1)
            train_i_y_slice.shape = (1)                           # Reshape training set output (1, 1)
            train_x[k-1,:] = train_i_x_slice
            train_y[k-1] = train_i_y_slice
     
    # Load test data
    test_set       = np.loadtxt("test_FD001.txt")
    test_set_x_org = test_set[:,sel]                                 # Test set input space (x)
    test_set_c     = test_set[:,np.array([1])]                       # Test set cycles (c)
    test_y         = np.loadtxt( "RUL_FD001.txt")    # Test set RUL (c)
    test_y.shape   = (test_y.shape[0], 1)
    
    # Normalize the data
    test_set_x = 2 * (test_set_x_org - lb) / (ub - lb) - 1   
    
    m_ts, N_en_ts = sliding_window(test_set, N_tw, stride)           # Number of training data & engines
    
    test_x = np.empty((N_en_ts, N_tw, N_ft), float)
    
    k = 0
    for ii in range(N_en_ts):
        engine         = test_set[:,0] == ii+1                       # Index for engine number 'i'
        test_i_x       = test_set_x[engine,:]                        # Engine 'i' test  data
        test_i_x_slice = test_i_x[-N_tw:]                          # Training input data for engine 'i' on time window 'h'
        test_i_x_slice.shape = (N_tw, N_ft)                       # Reshape training set input (N_tw, N_ft, 1)
        test_x[ii,:] = test_i_x_slice
    
    return train_x, train_y, test_x, test_y

In [25]:
X_train, Y_train, X_test, Y_test = load_dataset(N_tw, stride, sel, R_early)
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 17731
number of test examples = 100
X_train shape: (17731, 30, 14)
Y_train shape: (17731,)
X_test shape: (100, 30, 14)
Y_test shape: (100, 1)


# 2  Network Structure

The proposed deep learning method consists of two sub-structures, i.e. 5-layer LSTM networks and fully-connected layer for regression.

First, the input data sample is prepared in 2-dimensional (2D) format. The dimension of the input is $N_{tw} × N_{ft}$, where $N_{tw}$ denotes the time sequence dimension and $N_{ft}$ is the number of selected features (i.e. number of sensor measurements).

Next, 5 identical LSTM layers are stacked in the network for feature extraction. Each layer consists of 42 neurons and the activation function is relu.

Afterwards, the 2-dimensional feature map is connected with a fully-connected layer. Finally, one neuron is attached at the end of the proposed network for $RUL$ estimation.

**Model Hyperparameters:**

In [26]:

# Activation
activ = 'relu'

# 3  Model in Keras
Keras uses a different convention with variable names than TensorFlow. In particular, rather than creating and assigning a new variable on each step of forward propagation such as X, Z1, A1, Z2, A2, etc. for the computations for the different layers, in Keras code each line above just reassigns X to a new value using X = .... In other words, during each step of forward propagation, we are just writing the latest value in the commputation into the same variable X. The only exception was X_input, which we kept separate and did not overwrite, since we needed it at the end to create the Keras model instance (model = Model(inputs = X_input, ...) above).

In [27]:
def LSTM_2(input_shape,activ, layer):
    """
    Implementation of the 1D_CNN model.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
      
    # Define the input placeholder as a tensor with shape input_shape    
    X_input = Input(input_shape)
    
    X = LSTM(layer,return_sequences=True, activation=activ)(X_input)
    X = LSTM(layer,return_sequences=True,activation=activ)(X)
    X = LSTM(layer,return_sequences=True,activation=activ)(X)
    X = LSTM(layer,return_sequences=True,activation=activ)(X)
    X = LSTM(layer)(X)   

    X = Dense(100, activation='relu', name='fc')(X)
    X = Dense(1, name='RUL')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='CNN_2d')    
   
    return model

Now, we feed the model hyperparameters to the LSTM model

In [28]:
# Call the model
LSTM_2d = LSTM_2(X_train.shape[1:], activ, 42)

## 3.1 Model Summary

In [29]:
LSTM_2d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30, 14)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 30, 42)            9576      
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 42)            14280     
_________________________________________________________________
lstm_3 (LSTM)                (None, 30, 42)            14280     
_________________________________________________________________
lstm_4 (LSTM)                (None, 30, 42)            14280     
_________________________________________________________________
lstm_5 (LSTM)                (None, 42)                14280     
_________________________________________________________________
fc (Dense)                   (None, 100)               4300      
__________

## 3.2 Model Training

Its configuration is determined including the number of hidden layers and length etc. The LSTM takes as the inputs the normalized training data, and the labeled RUL values for the training samples are used as the target outputs of the network

The optimization of the network's weights was carried out with mini-batch stochastic gradient descent (SGD) and with the Adam algorithm. Xavier initializer is used for the weight initializations. The learning rate was set to 0.001 and was kept constant for the whole 50 epochs.

In [30]:
LSTM_2d.compile(optimizer = "Adam", loss = "mean_squared_error")

** Learning rate:**

In [31]:
print('Learning Rate: ' + str(K.get_value(LSTM_2d.optimizer.lr)))

Learning Rate: 0.001


We define an updatable plot to track training evolution

In [32]:
# updatable plot
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []        
        self.fig = plt.figure()        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, np.sqrt(self.losses), label="loss")
        plt.plot(self.x, np.sqrt(self.val_losses), label="val_loss")
        plt.ylabel('loss - RMSE')
        plt.xlabel('epoch')
        plt.legend(['train','test'], loc='upper left')
        plt.title('model loss = ' + str(min(np.sqrt(self.val_losses))))
        plt.show();
        
plot_losses = PlotLosses()

In [ ]:
LSTM_2d.fit(X_train, Y_train, epochs = 50, batch_size = 512, validation_data = (X_test, Y_test), callbacks=[plot_losses])

** Learning rate:**

In [34]:
print('Learning Rate: ' + str(K.get_value(LSTM_2d.optimizer.lr)))

Learning Rate: 0.001


Save the model (architecture, weights, ...) 

In [38]:
LSTM_2d.save('5-layer-LSTM.h5') 

# 4 Experimental Results

## 4.1 Performance Metrics
Three metrics were considered to compare our results: time of training, RMSE and the scoring function proposed in NASAdata. The scoring function $s$ is defined as follows.

\begin{align} \label{eq:someequation}
 s &= \sum_{i=1}^{N_{s}} exp(\alpha|\Delta_i|) 
\end{align}
Here $N_s$ denotes the total number of data samples, $\Delta_i$ is the difference between the estimated and real RUL and $\alpha$ is $\frac{1}{13}$ if we under-estimate and $\frac{1}{10}$ otherwise. Thus, this metric penalizes over-estimation more than under-estimation.

We use the standard definition of the root-mean-square error (RMSE).


In [ ]:
def score_cal(y_hat, Y_test):
    d   = y_hat - Y_test
    tmp = np.zeros(d.shape[0])
    for i in range(d.shape[0]):
        if d[i,0] >= 0:
           tmp[i] = np.exp( d[i,0]/10) - 1
        else:
           tmp[i] = np.exp(-d[i,0]/13) - 1
    return tmp 

Another popular metric to evaluate the effectiveness of the proposed method is Root Mean Square Error (RMSE).

## 4.2 Training set

In [39]:
preds = LSTM_2d.evaluate(x = X_train, y = Y_train)
print()
print ("Test  MSE = " + str(preds))
print ("Test RMSE = " + str(np.sqrt(preds)))
#

17731/17731 [==============================] - 31s 2ms/step
()
Test  MSE = 156.917735729
Test RMSE = 12.5266809542


In [ ]:
y_hat_tr   = LSTM_2d.predict(x = X_train)
#score_i_tr = score_cal(y_hat_tr, Y_train)
#score_tr   = print("Score = " + str(sum(score_i_tr)))

### 4.2.1 Plots

In [ ]:
d_tr = y_hat_tr - Y_train
plt.hist(d_tr, bins='auto')  
plt.title('Error distribution - Training Set')
plt.ylabel('f')
plt.xlabel("Error: $RUL_{hat}$ - RUL")
plt.show()

## 4.3 Test set

In [ ]:
y_hat   = LSTM_2d.predict(x = X_test)
#score_i = score_cal(y_hat, Y_test)
#score   = print("Score = " + str(sum(score_i)))

In [42]:
preds = LSTM_2d.evaluate(x = X_test, y = Y_test)
print()
print ("MSE = " + str(preds))
print ("RMSE = " + str(np.sqrt(preds)))

100/100 [==============================] - 0s 2ms/step
()
MSE = 191.373088989
RMSE = 13.8337662619


### 4.3.1 Plots

In [ ]:
d = y_hat - Y_test
plt.hist(d, bins='auto')  
plt.title('Error distribution - Test Set')
plt.ylabel('f')
plt.xlabel("Error: $RUL_{hat}$ - RUL")
plt.show()

In [ ]:
x     = range(0,100)
y_ts  = np.sort(Y_test[:,0])
idx   = np.argsort(Y_test[:,0])
y_tr  = y_hat[idx,0]
plt.plot(x, y_tr, 'bo-', x, y_ts, 'ro-')
plt.title('RUL vs. engine #')
plt.ylabel('RUL')
plt.xlabel('engine #')
plt.legend(['Prediction', 'Target'], loc='upper left')
plt.show()

In [ ]:
plt.plot(Y_test, y_hat, 'bo')
plt.plot(Y_test,Y_test, 'r-')
plt.plot(Y_test,Y_test+20, 'r--')
plt.plot(Y_test,Y_test-20, 'r--')
plt.title('RUL vs. RUL #')
plt.ylabel('RUL Estimated')
plt.xlabel('RUL True')
plt.show()

# 5 Conclusions

In [ ]:
K.tensorflow_backend._get_available_gpus()

In [43]:
Y_test[Y_test > R_early] = R_early                     # R_early = 125 
preds = LSTM_2d.evaluate(x = X_test, y = Y_test)
print()
print ("MSE = " + str(preds))
print ("RMSE = " + str(np.sqrt(preds)))

100/100 [==============================] - 0s 2ms/step
()
MSE = 157.928279419
RMSE = 12.5669518746


In [ ]:
x     = range(0,100)
y_ts  = np.sort(Y_test[:,0])
idx   = np.argsort(Y_test[:,0])
y_tr  = y_hat[idx,0]
plt.plot(x, y_tr, 'bo-', x, y_ts, 'ro-')
plt.title('RUL vs. engine #')
plt.ylabel('RUL')
plt.xlabel('Sorted engine #')
plt.legend(['Prediction', 'Target'], loc='upper left')
plt.show()

In [ ]:
plt.plot(Y_test, y_hat, 'bo')
plt.plot(Y_test,Y_test, 'r-')
plt.plot(Y_test,Y_test+20, 'r--')
plt.plot(Y_test,Y_test-20, 'r--')
plt.title('RUL vs. RUL #')
plt.ylabel('RUL Estimated')
plt.xlabel('RUL True')
plt.show()

In [ ]:
d = y_hat - Y_test
plt.hist(d, bins='auto')  
plt.title('Error distribution - Test Set')
plt.ylabel('f')
plt.xlabel("Error: $RUL_{hat}$ - RUL")
plt.show()

In [ ]:
max(Y_train)

In [ ]:
max(y_hat)

In [ ]:
plt.hist(Y_train, bins=20)  
plt.title('RUL distribution - Training Set')
plt.ylabel('f')
plt.xlabel("RUL")
plt.show()

In [ ]:
plt.hist(Y_test, bins=20)  
plt.title('RUL distribution - Test Set')
plt.ylabel('f')
plt.xlabel("RUL")
plt.show()

In [ ]:
plt.hist(y_hat, bins=20)  
plt.title('RUL distribution - Test Set')
plt.ylabel('f')
plt.xlabel("RUL")
plt.show()